# Region of Interst Time-Series Data

This notebook demonstrates the extraction of time-series for a Region of Interest (ROI). These requests return csv type files. We will post an ROI as GeoJSON to the API, and download a Planetary Variable (PV) time series.

Reference information can be found at [Data access - Requesting time-series](https://docs.vandersat.com/VanderSat_API_UserGuide.html#requesting-time-series)

Other API-endpoint operations can be found at [REST API - Planet Planetary Variables products](https://maps.vandersat.com/api/v2/)

In [1]:
import os
import requests

## Authenticating 

In [2]:
# Username and API Key stored as an environment variable
AUTH = (os.getenv("VDS_USER"), os.getenv("VDS_PASS"))

In [3]:
BASE_URL = "https://maps.vandersat.com/api/v2"

## Requests example
First, we will check if our authentication is working. We want a response code of `200` from this API call.

In [4]:
# Authenticate
session = requests.Session()
session.auth = AUTH

# Request
response = session.get(BASE_URL)
response

<Response [200]>

## ROI operation examples 
We can upload a ROI, to an account and receive PV time-series. First, let's list all the current ROIS inside the account.

In [5]:
ROIS_URL = os.path.join(BASE_URL, "rois")

In [6]:
# GET ROIS list
response = session.get(ROIS_URL)
response.json()["rois"]

[{'id': '765322',
  'name': 'Test',
  'description': None,
  'created_at': '2023-02-02T10:23:42.843461',
  'area': 233675478.05986023,
  'labels': []},
 {'id': '765326',
  'name': 'my_first_roi',
  'description': 'practice',
  'created_at': '2023-02-02T16:31:52.816605',
  'area': 368423.4808397293,
  'labels': []},
 {'id': '765327',
  'name': 'my_first_roi',
  'description': 'practice',
  'created_at': '2023-02-02T16:33:34.450132',
  'area': 368423.4808397293,
  'labels': []},
 {'id': '765329',
  'name': 'my_first_roi',
  'description': 'practice',
  'created_at': '2023-02-02T16:35:35.520576',
  'area': 368423.4808397293,
  'labels': []},
 {'id': '765328',
  'name': 'my_first_roi',
  'description': 'practice',
  'created_at': '2023-02-02T16:33:35.279605',
  'area': 368423.4808397293,
  'labels': []}]

Good chance there are no ROIS in the account yet, so let's create one first. Make sure the ROI is inside your account's area-allowed.

In [7]:
# Create geometry of ROI in the Netherlands
roi_geojson = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          4.909772872924805,
          51.989637331792146
        ],
        [
          4.922990798950195,
          51.989637331792146
        ],
        [
          4.922990798950195,
          51.99328417001946
        ],
        [
          4.909772872924805,
          51.99328417001946
        ],
        [
          4.909772872924805,
          51.989637331792146
        ]
      ]
    ]
}

# Name & describe the ROI
roi = {
    "name": "first_roi",
    "description": "practice",
    "geojson": roi_geojson,
}

# You can post multiple ROIS
rois = {
    "rois": [roi]
}

In [8]:
# POST the ROI
response = session.post(ROIS_URL, json=rois)
response.json()

{'rois': [{'id': '765567',
   'name': 'first_roi',
   'description': 'practice',
   'geojson': {'type': 'Polygon',
    'coordinates': [[[4.909772873, 51.989637332],
      [4.922990799, 51.989637332],
      [4.922990799, 51.99328417],
      [4.909772873, 51.99328417],
      [4.909772873, 51.989637332]]]},
   'created_at': '2023-02-08T20:57:45.312608',
   'deleted_at': None,
   'area': 368423.4808397293,
   'display': False,
   'metadata': [],
   'labels': []}]}

We can find our new ROI in the account's ROIS list.

In [9]:
# GET ROIS list
response = session.get(ROIS_URL)
roi_list = response.json()["rois"]

## Product operation examples
Now that we posted our ROI, our system will automatically calculate the time-series for all PVs available in the account. Note, the processing can take some time. We can request a list of the account's available PVs and see their information.

In [10]:
PRODUCT_URL = os.path.join(BASE_URL, "products")

In [12]:
response = session.get(PRODUCT_URL)
product_list = response.json()["products"]
product_list

[{'name': 'BIOMASS-PROXY_V2.0_10',
  'abbreviation': 'BIOMASS-PROXY_V2.0_10',
  'api_name': 'BIOMASS-PROXY_V2.0_10',
  'unit': None,
  'min_val': None,
  'max_val': None,
  'default_legend_id': '27',
  'max_zoom': 15,
  'time_series_type': None,
  'area_allowed': None,
  'groups': [],
  'field_based': True},
 {'name': 'TEFF-AMSR2-DESC_V4.0_1000',
  'abbreviation': 'TEFF-AMSR2-DESC_V4.0_1000',
  'api_name': 'TEFF-AMSR2-DESC_V4.0_1000',
  'unit': 'K',
  'min_val': 250.0,
  'max_val': 310.0,
  'default_legend_id': '2',
  'max_zoom': 15,
  'time_series_type': 'default',
  'area_allowed': None,
  'groups': [],
  'field_based': False}]

We can request a product's time-series for a ROI; we only need to define which `product_name`, and parameters `roi_id` & `start_time` & `end_time`.

In [13]:
product_name = product_list[1]["api_name"]  # e.g. 'TEFF-AMSR2-DESC_V4.0_1000'
roi_id = roi_list[0]["id"]  # e.g. '37999'
parameters = {
    "roi_id": roi_id,
    "start_time": "2012-07-25",
    "end_time": "2022-09-14",
    "provide_coverage": "true"
}

In [14]:
ROI_TS_URL = os.path.join(PRODUCT_URL, product_name, "roi-time-series-sync")
response = session.get(ROI_TS_URL, params=parameters)
response.content

b'# Data source: VanderSat\n# Creation date: 2023-02-01\n# Copyright: 2023, VanderSat B.V.\n# Documentation: https://docs.vandersat.com/\n# Contact information: https://vandersat.com/contact/\n# Unit K\ndate,TEFF-AMSR2-DESC_V4.0_1000,TEFF-AMSR2-DESC_V4.0_1000 COV\n2012-07-25,284.7025,100.0000\n2012-07-26,286.8975,100.0000\n2012-07-27,288.5850,100.0000\n2012-07-28,285.8394,100.0000\n2012-07-29,283.8382,100.0000\n2012-07-30,280.9585,100.0000\n2012-07-31,282.4399,100.0000\n2012-08-01,283.1315,100.0000\n2012-08-02,287.0931,100.0000\n2012-08-03,285.1182,100.0000\n2012-08-04,284.4575,100.0000\n2012-08-05,284.1266,100.0000\n2012-08-06,284.3486,100.0000\n2012-08-07,284.5565,100.0000\n2012-08-08,283.1568,100.0000\n2012-08-09,284.1525,100.0000\n2012-08-10,279.7983,100.0000\n2012-08-11,282.3296,100.0000\n2012-08-12,283.7959,100.0000\n2012-08-13,284.8229,100.0000\n2012-08-14,286.5400,100.0000\n2012-08-15,287.2004,100.0000\n2012-08-16,284.2376,100.0000\n2012-08-17,285.2759,100.0000\n2012-08-18,287.

The API responds the time-series text-csv format, which you can decode and read with, for instance, pandas. 

In [18]:
import pandas as pd
import io

# Read response
df = pd.read_csv(
    io.StringIO(response.content.decode("utf-8")),
    index_col=0,
    parse_dates=True,
    dayfirst=True,
    comment="#",
)
df

,TEFF-AMSR2-DESC_V4.0_1000,TEFF-AMSR2-DESC_V4.0_1000 COV
date,,
2012-07-25,284.7025,100.0
2012-07-26,286.8975,100.0
2012-07-27,288.5850,100.0
2012-07-28,285.8394,100.0
2012-07-29,283.8382,100.0
...,...,...
2022-09-10,282.2104,100.0
2022-09-11,282.6317,100.0
2022-09-12,282.0301,100.0


In [19]:
# Write to csv
fname = "/Users/octave.lepinard/Desktop/training_notebooks/pv_time_series.csv"
df.to_csv(fname)